In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
chitchat_df = pd.read_pickle("./chitchat.pkl")

In [ ]:
chitchat_df.head()

In [ ]:
chitchat_df.drop(['prompt'], axis=1, inplace=True)

In [ ]:
chitchat_df


In [ ]:
cc_new = pd.DataFrame().assign(X=chitchat_df['query'])

In [ ]:
cc_new["Y"] = 0

In [ ]:
cc_new = cc_new[["Y" ,"X"]]

In [ ]:
cc_new.head()

In [ ]:
reddit_df = pd.read_pickle("./reddit.pkl")

In [ ]:
reddit_df.head()

In [ ]:
reddit_df.shape[0] - reddit_df['selftext'].isna().sum()

In [ ]:
reddit_df = reddit_df.loc[reddit_df['is_submission'] == False][["parent_body","body"]]

In [ ]:
reddit_df.head(50)

In [ ]:
reddit_df.rename(columns={"parent_body": "query", "body": "response"}, inplace=True)

In [ ]:
merged_qr_df = pd.concat([chitchat_df, reddit_df])

In [ ]:
merged_qr_df

In [ ]:
reddit_new = pd.DataFrame().assign(X=reddit_df['parent_body'])

In [ ]:
reddit_new = pd.DataFrame().assign(X=reddit_df['query'])

In [ ]:
reddit_new["Y"] = 1

In [421]:
reddit_new = reddit_new[["Y", "X"]]

In [423]:
reddit_new

,Y,X
0,1,That’s what I thought. I’m sorry things haven’...
1,1,Nah i really don't care enough about this.
2,1,Let’s hear your counter-argument then.
3,1,Yeah i don't support the premise anyways: i do...
4,1,"Your premise is invalid, which invalidates you..."
...,...,...
33,1,People have been on this earth for thousands o...
34,1,So I was wasting time on Google Maps as usual ...
35,1,So I was wasting time on Google Maps as usual ...
36,1,Not really. Populations in those areas were mu...


In [ ]:
reddit_new.tail(30)

In [ ]:
reddit_df.loc[reddit_df['is_submission'] == False]["body"]

In [ ]:
# reddit_df['selftext'].isna().sum()
val = reddit_df['body'].isna().sum()
val

In [ ]:
reddit_new = pd.DataFrame().assign(X=reddit_df['selftext'])


In [424]:
res_df = pd.concat([reddit_new, cc_new])
res_df.drop_duplicates(inplace=True)

In [425]:
res_df

,Y,X
0,1,That’s what I thought. I’m sorry things haven’...
1,1,Nah i really don't care enough about this.
2,1,Let’s hear your counter-argument then.
3,1,Yeah i don't support the premise anyways: i do...
4,1,"Your premise is invalid, which invalidates you..."
...,...,...
166163,0,12 miles is solid.
166164,0,"Oh man that's too bad, I'll take lots of pictu..."
166165,0,"Hey as long as you have water and time, the hi..."
166166,0,"That is so true, I hear you can pay like three..."


In [426]:
import collections
from nltk.stem import PorterStemmer
import re
import contractions

In [427]:
data = []
def remove_emojis(data):
        emoj = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002500-\U00002BEF"  # chinese char
            u"\U00002702-\U000027B0"
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U0001f926-\U0001f937"
            u"\U00010000-\U0010ffff"
            u"\u2640-\u2642" 
            u"\u2600-\u2B55"
            u"\u200d"
            u"\u23cf"
            u"\u23e9"
            u"\u231a"
            u"\ufe0f"  # dingbats
            u"\u3030"
                        "]+", re.UNICODE)
        return re.sub(emoj, '', data)

for index, rows in res_df.iterrows():
    text = rows["X"]
    text = contractions.fix(text)
    if "&gt;" in text:
       text = text.replace("&gt;", "")
    if "&lt:" in text:
        text = text.replace("&lt;", "")

    regex_url = r"(http[^\s]+)"
    wo_url = re.sub(regex_url,"",text)
    wo_url = re.sub(r"#","",wo_url)
    # wo_url = re.sub(r"(@[^\s]+)","",wo_url)
    wo_url = re.sub(r"(@[^\s]+)","@USER_",wo_url)

    all_emoticons = [':/',':l',':-)','-.-',':::---))))','&lt;',":'(",'-_-',':)','xx','xxx',';-)',':(',':-/',':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
            ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D','d:',':-l','&lt;3',
            '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',':|','o_o',';)',
            'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',':-0','-__-',':(((',
            '<3'':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
            ':-[', ':-<', '=/', '>:(', ':(', '>.<', ":'-(", ":'(",  ':-c',
            ':c', ':{',  ';(', "=\\", ">:\\", ':\\']
        
    a = wo_url
    for i in all_emoticons:
        a = a.replace(i,'')
        a = remove_emojis(a)
        a = " ".join(a.split())

    data.append(a)  
    # print(wo_url)

In [429]:
data_y = []
for index, rows in res_df.iterrows():
    data_y.append(rows["Y"])

In [430]:
new_res = pd.DataFrame({"Y": data_y, "X":data})
# new_res.columns = ["X", "Y"]
# new_res = new_res[["Y", "X"]]

In [431]:
new_res

,Y,X
0,1,That is what I thought. I am sorry things have...
1,1,Nah i really do not care enough about this.
2,1,Let us hear your counter-argument then.
3,1,Yeah i do not support the premise anyways: i d...
4,1,"Your premise is invalid, which invalidates you..."
...,...,...
163690,0,12 miles is solid.
163691,0,"Oh man that is too bad, I will take lots of pi..."
163692,0,"Hey as long as you have water and time, the hi..."
163693,0,"That is so true, I hear you can pay like three..."


In [420]:
new_res.to_csv(r'A:\MS2021\SUNY BUFFALO\Third sem\Info Retrieval\Project 4\ir_project_4\backend\classifier\data.csv', header=None, index=None, sep=' ', mode='a')

In [ ]:
def classify_0_1(text):
    # text is a str
    smallWords = stopwords.words('english')
    # wordlist = list(smallWords.split( ))
    # n = len(wordlist)
    
    return 0

    

In [ ]:
len(stopwords.words('english'))

In [ ]:
classify_0_1("Hello")